In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import numpy as np
import pandas as pd
df = pd.read_csv('lung cancer survey.csv')
df_no_na = df.dropna()

df_age = df_no_na[df_no_na["AGE"] > 21]
# Use df_age for all models, where clustering algorithm models, such as Kmeans, randomforest, decision tree as they would have already categorise an age threshold within the model
df_age

# Use df_cluster only for logistic regression, lasso, ridge and elastic net - cross comparison of these models with df_age
df_cluster = df_age.copy()
df_cluster['cluster'] = df_cluster['AGE'].apply(lambda x: 1 if x >= 61 else 0)
df_cluster = df_cluster.drop("AGE", axis = 1)
df_cluster

,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER,cluster
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
1,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
8997,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1
8998,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
8999,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1


In [8]:
from copulas.multivariate import GaussianMultivariate
import numpy as np
import pandas as pd


# Fit a Gaussian copula to the data
copula = GaussianMultivariate()
copula.fit(df_age)

# Sample new data from the copula model
sampled_data = copula.sample(100)

# Display the sampled data
print(sampled_data.head())
data

/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:716: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:721: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:716: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:721: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))
/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/scipy/

     GENDER        AGE   SMOKING  YELLOW_FINGERS   ANXIETY  PEER_PRESSURE  \
0 -0.021273  56.528404  1.021190        0.119936  1.073951       1.047710   
1  1.177627  63.577309  1.025648        0.950963  0.052930       0.984588   
2 -0.045941  73.058061  0.936793        0.906847  0.090537       0.929773   
3  0.155282  75.807600 -0.021168        0.040194 -0.090821      -0.105751   
4  0.890285  42.961563  0.147730        1.012332  0.922256      -0.062249   

   CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0         0.943384  0.896457  1.002232 -0.040844          -0.099346  0.984834   
1        -0.039817 -0.033119  0.070862  1.061811           0.970758  0.047785   
2         0.959546 -0.034469 -0.021186  0.075817           0.970225  1.099979   
3         0.057269  0.971122  0.034516  1.110595           1.082656  1.040395   
4        -0.029937  0.013283  1.037996  0.141989           1.020849  0.864955   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHE

,binary_var,numeric_var
0,0,0.331222
1,1,-0.767776
2,1,0.225982
3,1,0.947009
4,0,-0.639231
...,...,...
995,0,0.205862
996,1,-1.051385
997,0,1.849930
998,1,-0.353683


In [12]:
?GaussianMultivariate

Init signature: GaussianMultivariate(*args, **kwargs)
Docstring:     
Class for a multivariate distribution that uses the Gaussian copula.

Args:
    distribution (str or dict):
        Fully qualified name of the class to be used for modeling the marginal
        distributions or a dictionary mapping column names to the fully qualified
        distribution names.
File:           ~/anaconda3/lib/python3.11/site-packages/copulas/multivariate/gaussian.py
Type:           type
Subclasses:     